<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [91]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.55 s


In [92]:
import math
import numpy as np
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == 'EUC_2D':
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 19.7 ms


In [93]:
lines = open("ulysses22.tsp", "r").readlines()
index_start_coordinates = 0
cont = 0
V = []

for line in lines:
  cont += 1
  if line.replace(" ", "").startswith("EOF"):
    break
  elif line.replace(" ", "").startswith("DIMENSION"):
    n = int(line.split(":")[1][1:])
  elif line.replace(" ", "").startswith("EDGE_WEIGHT_TYPE"):
    t = line.replace(" ", "").split(":")[1][1:]
  elif line.replace(" ", "").startswith("NODE_COORD_SECTION"):
    index_start_coordinates = cont
  elif index_start_coordinates > 0:
    V.append((int(line.split()[0]) - 1, [float(line.split()[1]), float(line.split()[2])])) #(i, [x_value, y_value])

#n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
#t = lines[4].split()[1]

time: 12 ms


In [94]:
# global variables

def encode(S):
  encoded_string = ""
  for s in S:
    encoded_string += " " + str(s[0])
  return encoded_string

subsets = {}
counter = 0
subsets[encode(V)] = counter
counter += 1

d = np.zeros(shape = (n, 2 ** (n - 1))) # 2^(n-1)
phi = np.zeros(shape = (n, 2 ** (n - 1)))

time: 388 ms


In [95]:
import time
start = time.time()
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  S_index = subsets[encode(S)]
  if (time.time() - start) > 600: # max time: 10 min
    return None
  elif (len(S) == 1) & (S[0][0] == v): # Caso base: la soluzione è il peso dell’arco {v, 0}
    return weight(V[v][1], V[0][1], t)
  elif d[v, S_index] != 0: # Distanza già calcolata, ritorna il valore memorizzato
    return d[v, S_index] 
  else:  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    mindist = math.inf
    minprec = None
    temp = [i for i in S if i[0] != v] # S \ {v} 
    if encode(temp) not in subsets:
      global counter
      subsets[encode(temp)] = counter
      counter += 1
    for u in temp:
      dist = HK_VISIT(u[0], temp)
      if dist is None:
        break
      else:
        w = weight(u[1], V[v][1], t)
        if (dist + w) < mindist:
          mindist = dist + w
          minprec = u[0]
    d[v, S_index] = mindist
    phi[v, S_index] = minprec
    return mindist

time: 17.2 ms


In [96]:
print(HK_VISIT(0, V))

7188.0
time: 9min 59s
